In [1]:
import requests
import pandas as pd
from os import path, makedirs
import re
from collections import OrderedDict
from datetime import datetime

In [5]:
annotation_all_types = [
    '-999999999',
    '-888888888',
    '-666666666',
    '-555555555',
    '-333333333',
    '-222222222',
    '*',
    -999999999,
    -888888888,
    -666666666,
    -555555555,
    -333333333,
    -222222222,
    -999999999.0,
    -888888888.0,
    -666666666.0,
    -555555555.0,
    -333333333.0,
    -222222222.0,
    '-999999999.0',
    '-888888888.0',
    '-666666666.0',
    '-555555555.0',
    '-333333333.0',
    '-222222222.0'
    ]

new_mexico_fips = {
    '001': 'Bernalillo County',
    '003': 'Catron County',
    '005': 'Chaves County',
    '006': 'Cibola County',
    '007': 'Colfax County',
    '009': 'Curry County',
    '011': 'De Baca County',
    '013': 'Doña Ana County',
    '015': 'Eddy County',
    '017': 'Grant County',
    '019': 'Guadalupe County',
    '021': 'Harding County',
    '023': 'Hidalgo County',
    '025': 'Lea County',
    '027': 'Lincoln County',
    '028': 'Los Alamos County',
    '029': 'Luna County',
    '031': 'McKinley County',
    '033': 'Mora County',
    '035': 'Otero County',
    '037': 'Quay County',
    '039': 'Rio Arriba County',
    '041': 'Roosevelt County',
    '043': 'Sandoval County',
    '045': 'San Juan County',
    '047': 'San Miguel County',
    '049': 'Santa Fe County',
    '051': 'Sierra County',
    '053': 'Socorro County',
    '055': 'Taos County',
    '057': 'Torrance County',
    '059': 'Union County',
    '061': 'Valencia County'
    }

mrcog_tracts = [
    '35001000107', '35001000108', '35001000109', '35001000113', '35001000110',
    '35001000111', '35001000112', '35001000115', '35001000116', '35001000118',
    '35001000119', '35001000206', '35001000114', '35001000117', '35001000120',
    '35001000121', '35001000122', '35001000123', '35001000124', '35001000125',
    '35001000127', '35001000128', '35001000126', '35001000129', '35001000203',
    '35001000204', '35001000205', '35001000401', '35001000707', '35001000708',
    '35001000710', '35001000711', '35001000712', '35001001500', '35001003400',
    '35001003501', '35001003714', '35001003718', '35001003721', '35001003722',
    '35001003723', '35001003724', '35001003725', '35001003726', '35001003731',
    '35001003732', '35001003733', '35001003803', '35001003804', '35001003807',
    '35001004715', '35001004722', '35001004738', '35001004742', '35001004749',
    '35001940500', '35001000207', '35001000208', '35001000300', '35001000402',
    '35001000501', '35001004734', '35001004736', '35001000502', '35001004740',
    '35001004741', '35001000601', '35001000603', '35001000604', '35001000704',
    '35001000713', '35001000801', '35001000901', '35001000714', '35001000903',
    '35001000904', '35001001101', '35001001102', '35001001200', '35001001300',
    '35001001400', '35001001600', '35001001700', '35001001800', '35001001900',
    '35001002000', '35001002100', '35001002200', '35001002300', '35001002401',
    '35001002402', '35001002500', '35001002600', '35001002700', '35001002900',
    '35001003001', '35001003002', '35001003100', '35001003202', '35001003201',
    '35001003502', '35001003707', '35001003717', '35001003600', '35001003712',
    '35001004001', '35001003715', '35001003719', '35001003728', '35001003729',
    '35001003730', '35001004604', '35001004727', '35001003735', '35001003736',
    '35001003737', '35001003738', '35001004746', '35001003805', '35001004716',
    '35001003806', '35001004300', '35001004401', '35001004402', '35001004501',
    '35001004502', '35001004602', '35001004603', '35001004712', '35001004713',
    '35001004717', '35001004720', '35001004723', '35001004724', '35001004726',
    '35001004725', '35001004728', '35001004729', '35001004733', '35001004739',
    '35001004753', '35001004735', '35001004737', '35001940600', '35001940700',
    '35001004743', '35001004745', '35001004747', '35001004744', '35001004748',
    '35001004750', '35001004751', '35001004752', '35043010714', '35043940700',
    '35043010503', '35043940300', '35043940500', '35043010601', '35043010702',
    '35043010900', '35043940200', '35043940600', '35043010602', '35043010703',
    '35043010705', '35043010712', '35043010713', '35043010715', '35043010716',
    '35043010717', '35043010720', '35043010723', '35043011000', '35043011200',
    '35043940900', '35043010718', '35043010722', '35043010719', '35043010721',
    '35043011100', '35061971300', '35049010310', '35049010312', '35049010311',
    '35057963201', '35057963202', '35057963600', '35057963700', '35061940300',
    '35061970102', '35061970901', '35061970902', '35061971100', '35061970101',
    '35061970200', '35061970301', '35061970302', '35061970800', '35061970303',
    '35061970401', '35061970404', '35061971400', '35061970405', '35061970700',
    '35061971000'
    ]

mrcog_block_groups = [
    '350010001071', '350010001072', '350010001081', '350010001082',
    '350010001091', '350010001092', '350010001102', '350010001103',
    '350010001111', '350010001112', '350010001121', '350010001132',
    '350010001202', '350010001222', '350010001235', '350010001101',
    '350010001122', '350010001131', '350010001151', '350010001161',
    '350010001141', '350010001142', '350010001152', '350010001153',
    '350010001162', '350010001172', '350010001181', '350010001171',
    '350010001191', '350010001215', '350010001182', '350010001192',
    '350010001213', '350010001214', '350010001201', '350010001211',
    '350010001212', '350010001221', '350010001223', '350010001231',
    '350010001232', '350010001234', '350010001224', '350010001233',
    '350010001241', '350010001243', '350010001262', '350010001292',
    '350010001242', '350010001244', '350010001272', '350010001251',
    '350010001253', '350010001252', '350010001273', '350010001283',
    '350010001261', '350010001271', '350010001291', '350010001281',
    '350010001282', '350010001284', '350010002031', '350010002032',
    '350010002042', '350010002052', '350010002054', '350010002062',
    '350010002071', '350010002082', '350010002083', '350010003004',
    '350010003005', '350010001293', '350010002041', '350010002073',
    '350010002051', '350010002053', '350010002061', '350010002081',
    '350010003001', '350010002072', '350010003002', '350010003003',
    '350010003006', '350010004021', '350010004023', '350010005011',
    '350010007071', '350010004011', '350010004012', '350010004013',
    '350010004022', '350010004024', '350010005012', '350010006011',
    '350010005021', '350010005022', '350010005023', '350010006012',
    '350010006013', '350010006014', '350010006031', '350010006033',
    '350010006041', '350010006042', '350010006032', '350010006043',
    '350010007041', '350010007042', '350010007043', '350010006044',
    '350010007072', '350010007074', '350010007102', '350010007112',
    '350010009011', '350010009015', '350010009031', '350010009032',
    '350010009034', '350010009042', '350010009043', '350010011011',
    '350010007073', '350010007075', '350010007082', '350010007083',
    '350010007081', '350010007101', '350010007103', '350010007111',
    '350010007113', '350010007122', '350010007121', '350010007132',
    '350010007133', '350010007134', '350010007131', '350010007141',
    '350010007142', '350010007143', '350010008011', '350010008012',
    '350010009012', '350010009014', '350010009016', '350010009017',
    '350010009033', '350010009041', '350010009013', '350010011014',
    '350010011015', '350010011023', '350010012002', '350010011012',
    '350010011013', '350010011021', '350010011022', '350010012001',
    '350010012003', '350010013003', '350010013004', '350010015001',
    '350010015002', '350010016002', '350010017005', '350010023003',
    '350010024012', '350010024021', '350010027002', '350010027005',
    '350010012004', '350010013002', '350010012005', '350010013001',
    '350010014001', '350010014002', '350010014003', '350010014004',
    '350010016001', '350010017001', '350010017002', '350010017003',
    '350010017004', '350010018001', '350010018002', '350010019001',
    '350010020001', '350010020002', '350010021001', '350010022001',
    '350010022002', '350010022003', '350010023001', '350010023002',
    '350010023004', '350010024011', '350010024013', '350010024022',
    '350010024024', '350010024023', '350010024025', '350010025001',
    '350010025002', '350010026001', '350010027001', '350010027003',
    '350010027004', '350010029001', '350010030011', '350010030012',
    '350010030022', '350010029002', '350010029003', '350010030013',
    '350010030014', '350010030021', '350010032011', '350010032012',
    '350010032023', '350010032024', '350010034001', '350010034002',
    '350010031001', '350010031002', '350010031003', '350010032021',
    '350010032022', '350010034003', '350010034004', '350010035021',
    '350010036002', '350010036004', '350010036005', '350010037071',
    '350010037072', '350010037122', '350010037141', '350010037151',
    '350010037152', '350010037153', '350010037154', '350010037171',
    '350010037172', '350010037174', '350010037182', '350010035011',
    '350010035012', '350010035013', '350010035022', '350010035023',
    '350010035024', '350010036001', '350010036003', '350010037073',
    '350010037121', '350010037123', '350010037124', '350010037142',
    '350010037143', '350010037144', '350010037173', '350010037181',
    '350010037191', '350010037192', '350010037211', '350010037213',
    '350010037222', '350010037231', '350010037232', '350010037234',
    '350010037235', '350010037241', '350010037281', '350010037291',
    '350010037292', '350010037193', '350010037233', '350010037212',
    '350010037214', '350010037221', '350010037242', '350010037251',
    '350010037252', '350010037253', '350010037243', '350010037261',
    '350010037262', '350010037282', '350010037283', '350010037284',
    '350010037301', '350010037302', '350010037303', '350010037312',
    '350010037322', '350010037352', '350010037371', '350010037372',
    '350010037381', '350010037382', '350010037383', '350010037384',
    '350010038031', '350010038032', '350010038041', '350010038042',
    '350010038043', '350010038051', '350010038052', '350010038071',
    '350010038072', '350010038073', '350010040011', '350010045021',
    '350010046033', '350010046043', '350010047121', '350010047132',
    '350010047133', '350010047161', '350010047201', '350010047202',
    '350010047222', '350010047233', '350010047251', '350010047252',
    '350010047272', '350010047282', '350010047332', '350010047341',
    '350010047351', '350010047361', '350010047381', '350010047391',
    '350010047411', '350010047412', '350010047413', '350010047423',
    '350010047431', '350010037311', '350010037321', '350010037331',
    '350010037323', '350010038061', '350010038062', '350010047382',
    '350010047383', '350010047392', '350010037351', '350010037361',
    '350010040012', '350010040013', '350010047372', '350010043001',
    '350010043002', '350010043003', '350010043004', '350010044011',
    '350010044012', '350010044021', '350010044022', '350010045011',
    '350010045012', '350010045022', '350010045023', '350010046021',
    '350010046022', '350010046031', '350010046032', '350010046041',
    '350010046042', '350010047122', '350010047403', '350010047131',
    '350010047331', '350010047342', '350010047401', '350010047402',
    '350010047151', '350010047421', '350010047152', '350010047171',
    '350010047172', '350010047173', '350010047221', '350010047223',
    '350010047231', '350010047232', '350010047234', '350010047241',
    '350010047242', '350010047261', '350010047281', '350010047271',
    '350010047283', '350010047291', '350010047292', '350010047422',
    '350010047333', '350010047352', '350010047343', '350010047424',
    '350010047371', '350010047432', '350010047441', '350010047442',
    '350010047453', '350010047471', '350010047472', '350010047481',
    '350010047482', '350010047483', '350010047491', '350010047492',
    '350010047501', '350010047502', '350010047503', '350010047511',
    '350010047521', '350010047522', '350010047523', '350010047531',
    '350010047532', '350019405001', '350019405002', '350019406001',
    '350019406002', '350019406003', '350019407001', '350010047451',
    '350010047452', '350010047461', '350010047462', '350019406004',
    '350430107181', '350430110001', '350430110002', '350439402002',
    '350430105031', '350430106011', '350439409001', '350439409002',
    '350430105032', '350430105033', '350430107152', '350430106012',
    '350430107031', '350430107034', '350430111001', '350439402001',
    '350439403001', '350430106021', '350430106022', '350430107021',
    '350430107022', '350430107032', '350430107023', '350430107024',
    '350439405001', '350430107033', '350430107051', '350430107052',
    '350430107053', '350430107054', '350430107121', '350430107193',
    '350430107122', '350430107123', '350430107124', '350430107131',
    '350430107132', '350430107133', '350430107134', '350430107151',
    '350430107141', '350430107142', '350430107161', '350430111004',
    '350430107162', '350430107163', '350430107164', '350430107171',
    '350430107211', '350430107172', '350430107173', '350430107202',
    '350430107222', '350430112001', '350430107191', '350430107192',
    '350430107201', '350430107203', '350430107212', '350430107233',
    '350430109002', '350439406001', '350439407001', '350430107221',
    '350430107231', '350430107232', '350430109001', '350430112002',
    '350430111002', '350430111003', '350439405002', '350439405003',
    '350490103111', '350490103121', '350490103101', '350490103112',
    '350490103122', '350579636005', '350579632011', '350579632021',
    '350579632022', '350579636002', '350579636003', '350579632012',
    '350579636001', '350579636004', '350579637002', '350619403001',
    '350619701025', '350619703013', '350619703014', '350619704041',
    '350619704051', '350619704052', '350619707002', '350619714002',
    '350579637001', '350619701011', '350619701012', '350619701023',
    '350619703031', '350619703032', '350619703034', '350619708004',
    '350619701013', '350619703021', '350619703022', '350619703033',
    '350619707001', '350619709021', '350619713001', '350619701021',
    '350619701022', '350619711001', '350619701024', '350619702003',
    '350619702001', '350619702002', '350619703011', '350619703012',
    '350619704011', '350619704012', '350619704013', '350619704042',
    '350619714001', '350619707003', '350619707004', '350619710001',
    '350619708001', '350619708002', '350619708003', '350619708005',
    '350619709011', '350619709012', '350619709013', '350619709014',
    '350619710002', '350619710003', '350619710004'
    ]

filename_dict = OrderedDict(
    {
        f'In {api_year} InflationAdjusted Dollars': '',
        ' ': '_',
        'The_United_States': 'U.S.',
        'In_The_Past_12_Months': 'In_Past_Year',
        'The_': '',
        '___': '_',
        '__': '_'
    }
)

In [3]:
def pandasRegion(DataFrame, SourceColumn, TargetColumn, Reference):
    def insideRegion(pd_df):
        g = pd_df[SourceColumn]
        if g in Reference:
            return 'Yes'
        else:
            return 'No'
    DataFrame[TargetColumn] = DataFrame.apply(insideRegion, axis=1)
    return DataFrame

def areaName(DataFrame, SourceColumn, TargetColumn, Reference):
    def nameLookup(pd_df):
        f = pd_df[SourceColumn]
        if f in Reference:
            n = Reference[f]
            return n
        else:
            return None
    DataFrame[TargetColumn] = DataFrame.apply(nameLookup, axis=1)
    return DataFrame

In [4]:
api_year = 2020
api_year = str(api_year)

In [13]:
acs_api_url = 'https://api.census.gov/data/{}/acs/acs5'

def mostRecentYear(program='acs5'):
    if program == 'acs5':
        year = datetime.now().year
        code = requests.get(acs_api_url.format(year)).status_code
        while code != 200:
            year -= 1
            check_url = acs_api_url.format(year)
            code = requests.get(check_url).status_code
        return year
    else:
        return None

tract_keywords = [
    'tract',
    'tracts'
]

block_group_keywords = [
    'blockgroup',
    'block_group',
    'block group',
    'bg',
    'blockgroups',
    'block_groups',
    'block groups',
    'bgs'
]

geography_keywords = tract_keywords + block_group_keywords

def setCounty(county_fips_code='*', state_fips_code='35'):
    state = str(state_fips_code).zfill(2)
    if state.isnumeric():
        state = '&in=state:{}'.format(state)
    else:
        return None
    ###################################################################################################################
    county_fips_code = str(county_fips_code)
    if county_fips_code.lower() in ['all', '*']:
        county = '&in=county:*'
    elif county_fips_code.zfill(3).isnumeric():
        county = '&in=county:{}'.format(county_fips_code.zfill(3))
    else:
        return None
    
    county_query = state + county
    return county_query

def setArea(area='tract', fips_code='*'):
    if area.lower() not in geography_keywords:
        return None
    
    if area.lower() in tract_keywords:
        area = 'tract'
    elif area.lower() in block_group_keywords:
        area = 'block%20group'
    else:
        return None
    
    fips_code = str(fips_code)
    if fips_code.lower() in ['*', 'all']:
        code = '*'
    elif fips_code.isnumeric():
        code = fips_code
    else:
        return None
    
    area_query = f'&for={area}:{code}'
    return area_query


def setTable(table):
    table = str(table)
    if '_' not in table:
        table = f'group({table})'
    elif '_' in table:
        table = table
    else:
        return None
    
    return table
    

def buildQuery(api_key, table, area_query=None, county_query=None, year=None):
    api_key = api_key

#     table = str(table)
#     if '_' not in table:
#         table = f'group({table})'
#     elif '_' in table:
#         table = table
#     else:
#         return None
    
    table = setTable(table)
    
    if year:
        year = str(year)
    else:
        year = mostRecentYear()
    
    if area_query:
        area_query = str(area_query)
        if '&for=' not in area_query:
            return None
        else:
            area_query = area_query
    else:
        area_query = setArea()

    if county_query:
        county_query = str(county_query)
        if '&in=' not in county_query:
            return None
        else:
            county_query = county_query
    else:
        county_query = setCounty()
    
    query = '{}?get={}{}{}&key={}'.format(
        acs_api_url.format(year),
        table,
        area_query,
        county_query,
        api_key
    )
    return query

In [8]:
mostRecentYear()

2020

In [10]:
setCounty()

'&in=state:35&in=county:*'

In [11]:
setCounty('043')

'&in=state:35&in=county:043'

In [14]:
setCounty(43)

'&in=state:35&in=county:043'

In [15]:
setArea()

'&for=tract:*'

In [16]:
mrcog_key = '722d2ba6712f3952d8daba907b503768680f20ce'

In [20]:
my_query = buildQuery(mrcog_key, 'B05002', setArea('tract'), setCounty('*'), api_year)

In [21]:
my_query

'https://api.census.gov/data/2020/acs/acs5?get=group(B05002)&for=tract:*&in=state:35&in=county:*&key=722d2ba6712f3952d8daba907b503768680f20ce'

In [25]:
def parseQuery(query):
    query_parts = query.split('=')
    api_parts = query_parts[0].split('/')
    year = api_parts[4]
    program = api_parts[6].split('?')[0]
    table = query_parts[1].split('&')[0]
    area = query_parts[2].split(':')[0]
    state = query_parts[3].split(':')[1].split('&')[0]
    county = query_parts[4].split(':')[0]
    key = query_parts[5]
    parsed_query = {
        'Year': year,
        'Program': program,
        'Table': table,
        'Area': area,
        'State': state,
        'County': county,
        'Key': key
    }
    return parsed_query
    
def createErrorLog():
    error_log = pd.DataFrame(
        {
        'Year': [],
        'Program': [],
        'Table': [],
        'Area': [],
        'State': [],
        'County': [],
        'Key': []
        }
    )
    return error_log

def getTable(query):
    call = requests.get(query)
    if call.status_code == 200:
        jsonArray = call.json()
        return jsonArray
    else:
        return None

def isEmptyTable(pandasDataFrame):
    shape_check = pandasDataFrame.dropna(axis='columns', how='all').shape[1]
    bg_check = 'block group' in pandasDataFrame.columns.to_list()
    if shape_check <= 5:
        return True
    elif shape_check == 6 and bg_check:
        return True
    else:
        return False
    
def getTableToPandas(query, error_log=None):
    result = getTable(query)
    if result:
        df = pd.DataFrame(result[1:], columns=result[0])
        ### NEED A BETTER CHECK THAN THIS ###
        ## if df.dropna(axis='columns', how='all').shape[1] in [5, 6]:
        # Make this a separate function, and/or make it a parameter
# ---
        if isEmptyTable(df):
#         if (
#             'tract' in query and df.dropna(axis='columns', how='all'.shape[1]) == 5
#         ) or (
#             'block%20group' in query and df.dropna(axis='columns', how='all').shape[1] == 6
#         ):
            if error_log:
                empty_result = parseQuery(query)
                # concat is not done in place, meaning the return needs to be captured and re-returned
                # .loc should do this in place
                # error_log = pd.concat([error_log, pd.DataFrame(empty_result)], ignore_index=True)
                error_log.loc[len(df.index)] = [v for k, v in empty_result.items()]
                return None
            else:
                return None
        else:
            return df
# ---
    else:
        bad_result = parseQuery(query)
        # error_log = pd.concat([error_log, pd.DataFrame(bad_result)], ignore_index=True)
        error_log.loc[len(df.index)] = [v for k, v in bad_result.items()]
        return None

In [22]:
parseQuery(my_query)

{'Year': '2020',
 'Program': 'acs5',
 'Table': 'group(B05002)',
 'Area': 'tract',
 'State': '35',
 'County': 'county',
 'Key': '722d2ba6712f3952d8daba907b503768680f20ce'}

In [23]:
errors = createErrorLog()
errors

,Year,Program,Table,Area,State,County,Key


In [26]:
isEmptyTable(errors)

True

In [27]:
errors

,Year,Program,Table,Area,State,County,Key


In [28]:
getTable(my_query)

[['B05002_001E',
  'B05002_001EA',
  'B05002_001M',
  'B05002_001MA',
  'B05002_002E',
  'B05002_002EA',
  'B05002_002M',
  'B05002_002MA',
  'B05002_003E',
  'B05002_003EA',
  'B05002_003M',
  'B05002_003MA',
  'B05002_004E',
  'B05002_004EA',
  'B05002_004M',
  'B05002_004MA',
  'B05002_005E',
  'B05002_005EA',
  'B05002_005M',
  'B05002_005MA',
  'B05002_006E',
  'B05002_006EA',
  'B05002_006M',
  'B05002_006MA',
  'B05002_007E',
  'B05002_007EA',
  'B05002_007M',
  'B05002_007MA',
  'B05002_008E',
  'B05002_008EA',
  'B05002_008M',
  'B05002_008MA',
  'B05002_009E',
  'B05002_009EA',
  'B05002_009M',
  'B05002_009MA',
  'B05002_010E',
  'B05002_010EA',
  'B05002_010M',
  'B05002_010MA',
  'B05002_011E',
  'B05002_011EA',
  'B05002_011M',
  'B05002_011MA',
  'B05002_012E',
  'B05002_012EA',
  'B05002_012M',
  'B05002_012MA',
  'B05002_013E',
  'B05002_013EA',
  'B05002_013M',
  'B05002_013MA',
  'B05002_014E',
  'B05002_014EA',
  'B05002_014M',
  'B05002_014MA',
  'B05002_015E',
  '

In [29]:
getTableToPandas(my_query, errors)

,B05002_001E,B05002_001EA,B05002_001M,B05002_001MA,B05002_002E,B05002_002EA,B05002_002M,B05002_002MA,B05002_003E,B05002_003EA,...,B05002_026MA,B05002_027E,B05002_027EA,B05002_027M,B05002_027MA,GEO_ID,NAME,state,county,tract
0,4336,None,-555555555,*****,4179,None,103,None,3183,None,...,None,0,None,13,None,1400000US35019961600,"Census Tract 9616, Guadalupe County, New Mexico",35,019,961600
1,4106,None,-555555555,*****,3964,None,90,None,2277,None,...,None,0,None,13,None,1400000US35059950200,"Census Tract 9502, Union County, New Mexico",35,059,950200
2,1970,None,626,None,1873,None,623,None,1232,None,...,None,22,None,26,None,1400000US35001004716,"Census Tract 47.16, Bernalillo County, New Mexico",35,001,004716
3,2896,None,318,None,2817,None,335,None,1588,None,...,None,0,None,13,None,1400000US35001000116,"Census Tract 1.16, Bernalillo County, New Mexico",35,001,000116
4,4171,None,551,None,4021,None,550,None,2272,None,...,None,0,None,13,None,1400000US35001000124,"Census Tract 1.24, Bernalillo County, New Mexico",35,001,000124
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
607,3316,None,544,None,3095,None,536,None,1714,None,...,None,0,None,13,None,1400000US35001000111,"Census Tract 1.11, Bernalillo County, New Mexico",35,001,000111
608,3073,None,591,None,2599,None,511,None,1885,None,...,None,0,None,13,None,1400000US35005000201,"Census Tract 2.01, Chaves County, New Mexico",35,005,000201
609,2184,None,458,None,1505,None,397,None,586,None,...,None,0,None,13,None,1400000US35013001807,"Census Tract 18.07, Doña Ana County, New Mexico",35,013,001807
610,2342,None,368,None,2015,None,372,None,1014,None,...,None,0,None,13,None,1400000US35025000704,"Census Tract 7.04, Lea County, New Mexico",35,025,000704


In [35]:
z = "0123456789"
# "0123456789".index('456')
# 4
z[:4]

'0123'

In [39]:
concept = "SEX BY OCCUPATION AND MEDIAN EARNINGS IN THE PAST 12 MONTHS (IN 2020 INFLATION ADJUSTED DOLLARS) FOR THE FULL-TIME, YEAR-ROUND CIVILIAN EMPLOYED POPULATION 16 YEARS AND OVER"

In [ ]:
'Inflation Adjusted'.upper(), 'Inflation Adjusted'.lower()

In [52]:
def removeInflation(string, space=' '):
    if space not in {' ', '_'}:
        return string
    else:
        s = string.lower().replace(' ', '_')
    
    inf_variation = [i for i in ['inflationadjusted', 'inflation-adjusted', 'inflation_adjusted'] if i in s]
    if inf_variation:
        start_i = s.index('_(in_')
        end_i = s.index('_dollars)') + len('_dollars)')
        shortened = s[:start_i] + s[end_i:]
        shortened = shortened.title()
    else:
        return string
    
    if space == ' ':
        shortened = shortened.replace('_', ' ')
    
    return shortened

#     if 'inflationadjusted' not in s and 'inflation adjusted' not in s:
#         return 'crap'
#     elif 'inflation adjusted' in s:
#         x = 1
#     else:
#         x = 0
#     inf_i = s.index('inflation')
#     start_i = inf_i - 9
#     end_i = inf_i + 26 + x
#     shortened = s[:start_i] + s[end_i:]
#     return shortened.title()

removeInflation(concept)

'Sex By Occupation And Median Earnings In The Past 12 Months For The Full-Time, Year-Round Civilian Employed Population 16 Years And Over'

In [45]:
if []:
    print('empty')
else:
    print('u know this')

u know this


In [51]:
'ABCD_EFGH'.lower().replace('_', ' ')

'abcd efgh'

In [53]:
v = 'https://api.census.gov/data/2020/acs/acs5/variables.json'
simpleVariableDict = requests.get(v).json()['variables']
simpleVariableDict

{'for': {'label': "Census API FIPS 'for' clause",
  'concept': 'Census API Geography Specification',
  'predicateType': 'fips-for',
  'group': 'N/A',
  'limit': 0,
  'predicateOnly': True},
 'in': {'label': "Census API FIPS 'in' clause",
  'concept': 'Census API Geography Specification',
  'predicateType': 'fips-in',
  'group': 'N/A',
  'limit': 0,
  'predicateOnly': True},
 'ucgid': {'label': 'Uniform Census Geography Identifier clause',
  'concept': 'Census API Geography Specification',
  'predicateType': 'ucgid',
  'group': 'N/A',
  'limit': 0,
  'predicateOnly': True,
  'hasGeoCollectionSupport': True},
 'B24022_060E': {'label': 'Estimate!!Total:!!Female:!!Service occupations:!!Food preparation and serving related occupations',
  'concept': 'SEX BY OCCUPATION AND MEDIAN EARNINGS IN THE PAST 12 MONTHS (IN 2020 INFLATION-ADJUSTED DOLLARS) FOR THE FULL-TIME, YEAR-ROUND CIVILIAN EMPLOYED POPULATION 16 YEARS AND OVER',
  'predicateType': 'int',
  'group': 'B24022',
  'limit': 0,
  'attr

In [60]:
table_list = ['B05002']

def shortVariableDict(variables, table_list):
    shortDict = {}
    for k, v in variables.items():
        if k.split('_')[0] in table_list:
            shortDict[k] = v
        else:
            continue
    return shortDict
            
def fullVariableDict(variables):
    attrs = {}
    for k, v in variables.items():
        for i in variables[k]["attributes"].split(','):
            if 'EA' in i:
                attrs[i] = v['label'].replace('Estimate!!', 'Annotation of Estimate!!')
            elif 'MA' in i:
                attrs[i] = v['label'].replace('Estimate!!', 'Annotation of Margin of Error!!')
            elif 'M' in i:
                attrs[i] = v['label'].replace('Estimate!!', 'Margin of Error!!')
    return attrs

shortenedVariables = shortVariableDict(simpleVariableDict, table_list)
allVariables = fullVariableDict(shortenedVariables)

In [61]:
allVariables

{'B05002_001EA': 'Annotation of Estimate!!Total:',
 'B05002_001M': 'Margin of Error!!Total:',
 'B05002_001MA': 'Annotation of Margin of Error!!Total:',
 'B05002_009EA': 'Annotation of Estimate!!Total:!!Native:!!Born outside the United States:',
 'B05002_009M': 'Margin of Error!!Total:!!Native:!!Born outside the United States:',
 'B05002_009MA': 'Annotation of Margin of Error!!Total:!!Native:!!Born outside the United States:',
 'B05002_008EA': 'Annotation of Estimate!!Total:!!Native:!!Born in other state in the United States:!!West',
 'B05002_008M': 'Margin of Error!!Total:!!Native:!!Born in other state in the United States:!!West',
 'B05002_008MA': 'Annotation of Margin of Error!!Total:!!Native:!!Born in other state in the United States:!!West',
 'B05002_007EA': 'Annotation of Estimate!!Total:!!Native:!!Born in other state in the United States:!!South',
 'B05002_007M': 'Margin of Error!!Total:!!Native:!!Born in other state in the United States:!!South',
 'B05002_007MA': 'Annotation of 

In [58]:
shortenedVariables

{'B05002_001E': {'label': 'Estimate!!Total:',
  'concept': 'PLACE OF BIRTH BY NATIVITY AND CITIZENSHIP STATUS',
  'predicateType': 'int',
  'group': 'B05002',
  'limit': 0,
  'attributes': 'B05002_001EA,B05002_001M,B05002_001MA'},
 'B05002_009E': {'label': 'Estimate!!Total:!!Native:!!Born outside the United States:',
  'concept': 'PLACE OF BIRTH BY NATIVITY AND CITIZENSHIP STATUS',
  'predicateType': 'int',
  'group': 'B05002',
  'limit': 0,
  'attributes': 'B05002_009EA,B05002_009M,B05002_009MA'},
 'B05002_008E': {'label': 'Estimate!!Total:!!Native:!!Born in other state in the United States:!!West',
  'concept': 'PLACE OF BIRTH BY NATIVITY AND CITIZENSHIP STATUS',
  'predicateType': 'int',
  'group': 'B05002',
  'limit': 0,
  'attributes': 'B05002_008EA,B05002_008M,B05002_008MA'},
 'B05002_007E': {'label': 'Estimate!!Total:!!Native:!!Born in other state in the United States:!!South',
  'concept': 'PLACE OF BIRTH BY NATIVITY AND CITIZENSHIP STATUS',
  'predicateType': 'int',
  'group':

In [59]:
'12345_001ma'[-2:]

'ma'

In [ ]:
def splitAnnotationTable(pandas_data_frame):
    df = pandas_data_frame
    